# Notebook 3 : application du modèle de NER, conversion en tableau CSV, et géolocalisation des adresses

Le notebook précédent nous a apporté comme données de sorties le texte extrait des images des colonnes, se trouvant dans `/sample_1951/pero`, dans divers formats (PNG, TXT, XML et XML-ALTO). Nous avons converti les XML en question en un unique fichier JSON-L grace au script `/notebooks_1951/XmlToJson.py` (attention à ne pas exporter les fichiers XML et XML-ALTO).

Nous avons procédé à l'annotation de 15 colonnes (c'est-à-dire 1160 lignes) pour le NER avec l'outil **Prodigy**. Nous avons ensuite utilisé Spacy pour entrainer un modèle de NER à partir de ces annotations : ce modèle se trouve à `modeles_1951/3_Prodigy/model-best`. Nous avons en réalité entrainé plusieurs modèles et comparé leurs performances (cf. le mémoire pour plus de détails)

Nous partons donc des données des annuaires en format JSON-L. Nous appliquons dans un premier temps le modèle de NER que nous avons entrainé (qui se trouve ici : `/modeles_1951/2_Prodigy/model-best`). Nous allons ensuite associer des coordonnées géographiques à chaque adresse, en utilisant le géocodeur du gouvernement. Finalement, nous convertissons ces données en tableau, ou les colonnes sont les étiquettes du NER, c'est-a-dire : le nom de la personne, son adresse, etc. Nous faisons deux tableaux différents : un ou une ligne du tableau correspond à un propriétaire (et son ou ses immeubles), et un ou une ligne correspond à un immeuble parisien. Nous exportons ensuite ces tableaux en format CSV.
- input : le contenu de l'annuaire (format : JSON-L)
- output : deux tableaux CSV :
    - un premier où une ligne = un propriétaire ;
    - un deuxième où une ligne = un immeuble.

Les temps indiqués sont pour le traitement des 531 pages de l'annuaire des propriétaires.


<br>

***

<br>

Nous nous servons d'un environnement avec :
- python version 3.10.14
- pandas version 2.1.3
- numpy version 1.26.4
- geopandas version 0.14.0
- requests version 2.31.0
- spacy version 3.7.4
- folium version 0.15.0

# Etape I : Application du NER ligne par ligne et mise en forme des entrées


## 1. Import des librairies, définition des chemins, et import des données

In [1]:
# import des libraries

# pour le NER
import spacy
import json
import csv
from collections import defaultdict
import re

# pour les dataframes et geodataframes
import pandas as pd
import geopandas
from shapely.geometry import Polygon, LineString, Point

# Libraries pour le geocodeur
import ast
import requests
import io
import os

# Pour les cartes
import folium
from folium.plugins import MarkerCluster
from folium import Map
from folium.plugins import HeatMap

# Desactiver le nombre max de colonnes
pd.set_option('display.max_columns', None)

# Pour montrer les images
%matplotlib inline

/home/aaron/anaconda3/envs/memoire/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
# Définition des paths intéressants
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(parent_dir)
path_df_rues = os.path.join(parent_dir, 'liste_adresses_1951/6_liste_rues_1951.csv')
print(path_df_rues)

/home/aaron/Documents/M2/memoire_M2/0_donnees_rangees
/home/aaron/Documents/M2/memoire_M2/0_donnees_rangees/liste_adresses_1951/6_liste_rues_1951.csv


In [3]:
# Chargement du dataframe qui associe chaque code rue au nom de la rue explicite. Dataframe propre car relu à la main.
df_codes = pd.read_csv(path_df_rues)

In [4]:
# On définit aussi une liste avec les étiquettes du NER (pour créer le CSV)
fieldnames = ["PER", "PRENOM", "STATUT", "ORG", "NUM", "TYPE_VOIE", "NOM_VOIE", "ARR", "VILLE", "LOC", "PART", "CODES"]

## 2. Définition et application de la fonction convertissant le JSON-L en CSV

Cette fonction prend en entrée des données textuelles (ici le contenu d'un annuaire de propriétaires), y applique un modèle de NER, puis crée un tableau CSV avec ces informations.

Les colonnes du CSV correspondent aux étiquettes du NER.

Dans le fichier CSV de sortie, une ligne correspond à une entrée de l'annuaire. Nous définissons le début d'une entrée de l'annuaire la première étiquette qui ne soit pas un CODES mais soit juste après un CODE. Et nous définissons la fin d'une entrée comme la dernière étiquette "CODES" (c'est-à-dire la première étiquette CODES qui ne soit pas suivie d'un CODES). Le but de cette définition est de prendre en compte tous les codes d'une personne dans une seule entrée, si ils sont répartis sur plusieurs lignes.

In [5]:
def json_2_csv_v2(input_path, output_path, model_path, fieldnames):

    '''
    Cette fonction prend en entrée des données textuelles (ici le contenu d'un annuaire de propriétaires),
    y applique un modèle de NER, puis crée un tableau CSV avec ces informations.
    Les colonnes du CSV correspondent aux étiquettes du NER.
    - input_path > path des données d'entrée. Doit etre de format JSON-L
    - output_path > path des données de sorties. Doit etre de format CSV
    - model_path > path du modele de NER Prodigy qu'on va utiliser
    - fieldnames > liste des étiquettes du NER. Correspondra aux colonnes du CSV.
    '''

    # Chargement des données
    with open(input_path, 'r', encoding='utf-8') as input_file:
        input_data = [json.loads(line) for line in input_file]

    # Chargement du modele Prodigy
    nlp = spacy.load(model_path)



    # Etape 1 : On applique le NER ligne par ligne
    liste_output_prodigy = []

    for input_item in input_data:

        # On crée un dictionnaire de sortie. Ce dico va garder les info de texte et metadonnes du jsonl, et rajouter les infos du NER
        dico_output = {}
        dico_prodigy_definitif = defaultdict(list)

        # On va chercher les éléments qui nous intéressent
        text = input_item['text']
        meta = input_item['meta']
        doc = nlp(text)

        # Dico temporaire ou on rajoute les couples texte-étiquette classés par le NER.
        # .append permet de conserver toutes les informations, quand une étiquette est associée à plusieurs chaines de
        # caractères dans une seule ligne
        dico_prodigy = defaultdict(list)
        for ent in doc.ents:
            dico_prodigy[ent.label_].append(ent.text)

        # Mais maintenant dans ce dictionnaire, chaque étiquette est associée à une liste
        # Des fois, cette liste ne contient qu'un seul élément
        # On va remplir un deuxième dictionnaire mais ne faire des listes que quand il y a effectivement plusieurs éléments,
        # sauf pour CODES ou c'est plus pratique de garder la structure de liste pour toutes les lignes
        for key, value in dico_prodigy.items():
            if key != "CODE":
                if len(value) == 1:
                    dico_prodigy_definitif[key] = value[0]

                if len(value) > 1:
                    dico_prodigy_definitif[key] = value


            # On fait en sorte que la colonne "CODES" contienne une liste de tous les CODES détectés par le NER
            
            if key == 'CODE':
                codes = []
                                        
                for index, item in enumerate(value):
                    codes.append(item)

                dico_prodigy_definitif["CODES"] = codes

        
        # On rajoute les informations qu'on veut dans le dico de sortie de cette ligne :
        # Les couples texte-étiquettes qu'on a trouvés et mis en forme plus tot, le texte original, les métadonnées

        dico_output['prodigy'] = dico_prodigy_definitif
        dico_output['text'] = text
        dico_output['meta'] = meta


        # Et on rajoute ce dictionnaire de sortie de la ligne dans une liste qui contiendra tous ces dictionnaires

        liste_output_prodigy.append(dico_prodigy_definitif)




    # Etape 2 : On reconstitue les entrées, pour avoir un dictionnaire = une entrée de l'annuaire (et plus = une ligne de l'annuaire)
    # une entrée = les étiquettes apres un code et jusqu'au code d'après.
    
    # Initialisation :
    liste_entrees = []

    code_n_1 = False
    code_n = False

    temp_dict = defaultdict(list)


    # On itère sur les items de la liste des dicos des lignes
    # Si on a un code dans l'entrée n-1 et pas dans l'entrée n, on le rajoute à la liste car c'est comme ca que je définis une entrée
    # Sinon, on rajoute uniquement l'item dans un dictionnaire temporaire

    for item in liste_output_prodigy:

        # On récupère l'info : est-ce que cet élément à pour étiquette CODES ?
        if "CODES" in item:
            code_n = True

        else:
            code_n = False

        # Cas 1 : n-1 est un CODES et n n'est pas un code
        # alors, la ligne n-1 est la dernière ligne d'une entrée
        # Dans ce cas, on rajoute les informations stockées dans le dico temporaire dans la liste des entrées
        # On récupère aussi la ligne n et on la rajoute dans le dico temporaire après l'avoir réinitialisé
        if (code_n_1 == True) and (code_n == False):
            liste_entrees.append(temp_dict)
            temp_dict = defaultdict(list)

            # Code pour rajouter correctement l'entrée dans la liste des entrées
            for item_key, item_value in item.items():
                # Cas A : la clef n'existe pas > pas de précaution nécessaire
                if item_key not in temp_dict.keys():
                    temp_dict[item_key] = item_value

                # Cas B : la clef existe
                else:
                    # On traite differemment "code" et le reste parce que on veut une grande liste avec tous les codes pour code,
                    # et pour les autres on veut juste les rajouter dans la liste

                    # Sous-cas 1 : Cette valeur est déjà une liste
                    if isinstance(temp_dict[item_key], list):
                        if item_key == "CODES":
                            temp_dict[item_key].extend(item_value)
                        else:
                            temp_dict[item_key].append(item_value)

                    # Sous-cas 2 : Cette valeur n'est pas une liste                        
                    else:
                        temp_dict[item_key] = [temp_dict[item_key]]
                        if item_key == "CODES":
                            temp_dict[item_key].extend(item_value)
                        else:
                            temp_dict[item_key].append(item_value)
                        

        # Cas 2 : nous n'avons pas "n-1 est un CODES et n n'est pas un code"
        # cas ou n-1 n'est pas la dernière ligne d'une entrée
        # On utilise un code semblable pour rajouter le contenu de la ligne n-1 dans le dico temporaire
        else:
            for item_key, item_value in item.items():
                if item_key not in temp_dict.keys():
                    temp_dict[item_key] = item_value
                else:
                    if isinstance(temp_dict[item_key], list):
                        if item_key == "CODES":
                            temp_dict[item_key].extend(item_value)
                        else:
                            temp_dict[item_key].append(item_value)

                    else:
                        temp_dict[item_key] = [temp_dict[item_key]]
                        if item_key == "CODES":
                            temp_dict[item_key].extend(item_value)
                        else:
                            temp_dict[item_key].append(item_value)


        # On fait passer les informations "est-ce que les lignes n et n-1 sont des CODES ?"
        # de n a n+1
        code_n_1 = code_n
        code_n = None

    # Cas 3 : c'est la dernière ligne de l'annuaire
    if item == liste_output_prodigy[-1]:
        liste_entrees.append(temp_dict)



    # Etape 3 : On sauvegarde ces informations dans le CSV de sortie

    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        for row in liste_entrees:
            writer.writerow(row)

    return liste_entrees

    


In [6]:
# Application à nos données

input_path = os.path.join(parent_dir, 'sample_1951/contenu_echantillon.jsonl')
output_path = os.path.join(parent_dir, 'sample_1951/contenu_echantillon.csv')
model_path = os.path.join(parent_dir, 'modeles_1951/2_Prodigy/model-best')

liste_entrees = json_2_csv_v2(input_path, output_path, model_path, fieldnames)

/home/aaron/.local/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.4.1 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [7]:
# > 4 min 30 de traitement (pour l'ensemble de l'annuaire)

# Etape 2 : utiliser ces resultats pour produire deux autre fichiers CSV :
- un CSV où une ligne = un étiquette CODES (c'est-a-dire "une rue : un ou plusieurs immeubles")
- un CSV où un immeuble = une ligne (très utile pour faire des cartes sur le nombre d'immeubles)

## 1. Conversion en dataframes des données, création d'un index propriétaire, et d'une colonne comptabilisant le nombre de codes par entrée

In [8]:
# Conversion des données en dataframe
df_proprios = pd.DataFrame(liste_entrees, columns=fieldnames)
# Normalisation des noms de colonnes (pour les mettre en minuscules)
dico_fieldnames = {item : item.lower() for item in fieldnames}
df_proprios = df_proprios.rename(dico_fieldnames, axis=1)

In [9]:
# Je cree un index pour le df, chaque propriétaire aura un index
index_total_data = pd.Series(range(0,len(df_proprios)))
df_proprios.insert(0, 'index_total', index_total_data)

In [10]:
# Je créé une colonne "nb_codes" qui compte le nombre de codes pour chaque entree
df_proprios['nb_codes'] = df_proprios['codes'].apply(len)

## 2. Définition et application d'une fonction pour récupérer les codes séparés sur deux lignes
On constate que certaines entrée ont des codes séparées sur deux lignes, de type '4923 :', '242'. Nous allons le solidariser en une seule chaine de caractères. Cette fonction fonctionne jusqu'à 8 numéros de rues différents (par exemple : '4923 :', '242, 244, 246, [...], 256')

In [11]:
def fix_codes(row):
    # On récupère tous les codes de l'entrée dans une liste
    liste_codes = row.codes
    nvelle_liste = []

    code_content_n = ""
    code_content_n_1 = ""
   
    # Cas particulier pour un seul element
    # Pas besoin de vérifier, le code est sur une seule ligne
    if len(liste_codes) == 1:
        nvelle_liste = liste_codes

    # Si n = 2 ou plus, c'est-a-dire si le code est sur + d'une ligne
    else:
        for index_code, code_content in enumerate(liste_codes):
            code_content_n = code_content
            
            regex_1 = r'\d{3,4}( )?:( \d{1,2}( bis)?( ter)?(,)?)?( \d{1,2}( bis)?( ter)?(,)?)?( \d{1,2}( bis)?( ter)?(,)?)?'
            regex_2 = r'\d{1,3}( bis)?( ter)?([,\.])?( \d{1,3}( bis)?( ter)?)?([,\.] \d{1,3}( bis)?( ter)?)?([,\.] \d{1,3}( bis)?( ter)?)?([,\.] \d{1,3}( bis)?( ter)?)?([,\.] \d{1,3}( bis)?( ter)?)?([,\.] \d{1,3}( bis)?( ter)?)?([,\.] \d{1,3}( bis)?( ter)?)?(\.)?'

            # Cas 1 : On a effectivement un code sur deux lignes de type : '4923 :', '242'
            if (re.fullmatch(regex_1, code_content_n_1)) and (re.fullmatch(regex_2, code_content_n)):
                bon_code = f"{code_content_n_1} {code_content_n}"
                nvelle_liste.append(bon_code)
                code_content_n_1 = ""
                code_content_n = ""

            # Cas 2 : Dernier code : ne peut pas avoir un code sur deux lignes avec la ligne d'apres
            elif index_code == (len(liste_codes)-1):
                if code_content_n_1 != "":
                    nvelle_liste.append(code_content_n_1)
                if code_content_n != "":
                    nvelle_liste.append(code_content_n)

            # Cas 3 : sinon
            else:
                if code_content_n_1 != "":
                    nvelle_liste.append(code_content_n_1)

                code_content_n_1 = code_content_n
                code_content_n = ""
  
    return nvelle_liste

In [12]:
df_proprios['nveaux_codes'] = df_proprios.apply(fix_codes, axis=1)

In [13]:
# 0.5 sec de traitement (pour toutes les données)

## 3. Création d'une deuxième dataframe ou une ligne = un code

In [14]:
# On copie le dataframe
df_bis = df_proprios

# On sauvegarde la colonne codes
df_bis['code'] = df_bis['nveaux_codes']

# Puis on splite
df_bis = df_bis.explode('code')

In [15]:
# 0.1 sec (pour toutes les données)

## 4. Définition et application de la fonction qui transforme les codes en adresses explicites
Fonction qui extrait la voie explicite qui correspond au code de la voie, et le ou les plusieurs numéros possédés dans cette rue

Par ex, si le code est "4923 : 109, 111", le code va :
- dire que "4923" correspond au "boulevard Voltaire" grace au df des rues importé plus haut
- découper les codes "109" et "111"

In [16]:
pattern = r'(\d{1,4})[^\d]*(\d{1,3})([^\d]*(\d{1,3}))?([^\d]*(\d{1,3}))?([^\d]*(\d{1,3}))?([^\d]*(\d{1,3}))?'

liste_erreurs = []

def d_code_voie(row):
    code = row['code']
    match = re.search(pattern, code)
    if match:
        # Si on trouve bien ce pattern
        # Alors on recupere les infos : code de la rue, numero/s d'immeuble/s
        groups = [match.group(1), match.group(2), match.group(4), match.group(6), match.group(8), match.group(10)]
        code_voie = groups[0]
        numeros = groups[1:]

        # On enleve aussi les None causes par le regex
        numeros = [x for x in numeros if x is not None]

        # Conversion code voie > nom explicite de la voie
        try:
            df_codes.loc[df_codes['code'] == code_voie, 'nom_voie'].values[0]
        except:
            liste_erreurs.append([row, code_voie])
        
        else:
            nom_voie = df_codes.loc[df_codes['code'] == code_voie, 'nom_voie'].values[0]
            type_voie = df_codes.loc[df_codes['code'] == code_voie, 'type_voie'].values[0]
            arr = df_codes.loc[df_codes['code'] == code_voie, 'arr'].values[0]

            return code_voie, numeros, len(numeros), type_voie, nom_voie, arr

## 5. Création d'un troisieme dataframe ou une ligne = un immeuble (ce qui nous intéresse pour l'index par immeuble)

In [17]:
df_ter = df_bis

nvelles_col = ["code_voie", "nums", "nb_nums", "type_voie_imm", "nom_voie_imm", "arrs_imm"]

df_ter[nvelles_col] = df_ter.apply(d_code_voie, axis=1, result_type='expand')

df_ter['num_imm'] = df_ter['nums']
df_ter = df_ter.explode('num_imm')

In [18]:
# 3 min de traitement (pour toutes les données)

Notes :
- on ne prend pas en compte quand un numéro est 'bis' ou 'ter'. Mais ça devrait etre assez proche géographiquement pour pas poser de probleme in fine (surtout à l'échelle d'un quartier)
- ce code ne prend pas en compte les "1 à 5" et "numéros impairs" parce qu'ils ne sont pas explicites dans les données, ce qui est dommage. (La raison est que c'est une opération très chronophage)

In [19]:
# On créé la colonne "nb_imms" qui compte combien d'immeubles possède chaque personne au total
def count_imms(row):
    return len(df_ter.loc[df_ter.index_total == row.index_total])

df_ter["nb_imms"] = df_ter.apply(count_imms, axis=1)

In [20]:
# un peu en dessous de 30 sec (pour toutes les données)

In [21]:
# On harmonise les colonnes pour utiliser les memes noms que dans les données de 1898

dico_noms_colonnes = {'per' : 'nom_pers', 'prenom' : 'prenom_pers', 'statut' : 'civilite_pers',
'num' : 'num_pers', 'type_voie' : 'type_voie_pers', 'nom_voie' : 'nom_voie_pers', 'arr' : 'arr_pers',
'ville' : 'ville_pers', 'loc' : 'loc_pers', 'part' : 'part_pers',
'code_voie' : 'code_voie_imm', 'nums' : 'liste_nums_imm', 'nb_nums' : 'nb_nums_imm'}

In [22]:
df_ter = df_ter.rename(mapper=dico_noms_colonnes, axis=1)

# Etape 3 : définition et application d'une fonction pour le géocodage des adresses des immeubles, et des adresses des domiciles des propriétaires
Nous allons géocoder nos données, c'est-à-dire associer à chaque adresse des coordonnées géographiques. Nous allons pour ce faire appliquer le géocodeur du gouvernement.

Note : un géocodeur historique (développé par Bertrand Duménieu) est également disponible, et nous avons travaillé par ailleurs sur une comparaison entre les deux géocodeurs sur nos données.

In [23]:
# fonction pour appliquer le géocodeur historique
def mass_search(df, col_numero, col_type, col_nom, 
                *, 
                suffix, col_ville=None):
    """
    appelle le géocodeur https://api-adresse.data.gouv.fr
    et renvoie le dataframe avec de nouvelles informations :
    latitude, longitude, ville et type de résultat de l'adresse trouvée
    
    Paramètres :
      df :
        le dataframe en entrée
      col_numero :
      col_type :
      col_nom :
        le nom des 3 colonnes qui contiennent le numéro, le type
        de voie et le nom de la rue qui seront utilisées par le
        géocodeur
      col_ville :
        ville dans laquelle le géocodeur va chercher, par défaut Paris
      suffix :
        est utilisé pour nommer les 4 colonnes en sorties
        par exemple suffix="imm" fera comme résultat :
        lat_imm, lng_imm, result_type_imm et result_city_imm
    
    """
    
    # constantes internes
    dunder_indexname = f'__index_{suffix}__'
    dunder_filename = '__search_data__.csv'

    # cherche le nom de l'index ; en créé un si inexistant
    if df.index.name is None:
        df.index.name = dunder_indexname
    indexname = df.index.name
    
    # vérifier que l'index est unique
    df.reset_index(inplace=True)

    # créer le dataframe qu'on enverra au géocodeur
    search_data = df[[col_numero, col_type, col_nom]]

    # si on a pas de col_ville
    col_ville = col_ville or 'Paris'
    # on remplit la colonne ville
    if col_ville in df.columns:
        # si la colonne ville est une colonne existante
        search_data['city'] = df[col_ville]
    else:
        # sinon, rajouter la constante (en général Paris)
        search_data['city'] = col_ville
        
    # sauvegarder les informations pour les envoyer au géocodeur
    search_data.to_csv(dunder_filename, index=False)
    with open(dunder_filename) as feed:
        result = requests.post(
            "https://api-adresse.data.gouv.fr/search/csv/", 
            files={'data': feed},
            data={'columns': search_data.columns})
    # sauvegarder le résultat
    result = pd.read_csv(io.StringIO(result.text))
    
    # on ne garde que ce qui nous intéresse
    result = (result[['latitude', 'longitude', 'result_type', 'result_city']]
              # on renomme pour respecter les suffixes
              .rename(columns={
                          'latitude': f'lat_{suffix}',
                          'longitude': f'lng_{suffix}',
                          'result_type': f'result_type_{suffix}',
                          'result_city': f'result_city_{suffix}',
                      }))
    
    # on fusionne le df initial et celui qu'on a obtenu du géocodeur
    # et on restore l'index initial
    return df.merge(result, left_index=True, right_index=True).set_index(indexname)


In [24]:
# Application de la fonction
geoloc = mass_search(df_ter, 'num_pers', 'type_voie_pers', 'nom_voie_pers', suffix="pers", col_ville='ville_pers')
geoloc = mass_search(geoloc, 'num_imm', 'type_voie_imm', 'nom_voie_imm', suffix="imm")

/tmp/ipykernel_47234/119662777.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search_data['city'] = df[col_ville]
/tmp/ipykernel_47234/119662777.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search_data['city'] = col_ville


In [25]:
# 14 min d'exécution (pour toutes les données)

# Etape 4 : Rajouter les colonnes de quartier
Nous travaillons à l'échelle du quartier parisien (unité administrative), nous avons donc besoin d'associer un quartier à chaque adresse. Pour ce faire, nous chargeons les coordonnées des quartiers depuis Open Data Paris (cf. https://opendata.paris.fr/explore/dataset/quartier_paris/map/?disjunctive.c_ar&location=20,48.8932,2.36984&basemap=jawg.streets) dans Geopandas. Nous regardons ensuite dans quel quartier se trouvent les coordonnées trouvées par le géocodeur.

## 1. On commence par les adresses des immeubles parisiens

In [26]:
# On transforme 'geoloc' en un geodataframe
geoloc['coord_imm'] = [Point(xy) for xy in zip(geoloc.lng_imm, geoloc.lat_imm)] 
geoloc = geopandas.GeoDataFrame(geoloc, geometry=geoloc.coord_imm) 
type(geoloc)

geopandas.geodataframe.GeoDataFrame

In [27]:
# Import des donnees quartiers d'Open Data Paris
csv_folder = os.path.join(parent_dir, 'donnees_quartiers')
gdf_quartiers = geopandas.read_file(os.path.join(csv_folder, 'quartier_paris.shp'), encoding='utf-8')

gdf_quartiers['c_qu'] = gdf_quartiers["c_qu"].astype(int)

In [28]:
# On va chercher dans quel quartier se trouve chaque adresse d'immeuble
df_av_quartiers = gdf_quartiers.sjoin(geoloc, predicate='contains', how='right')

/home/aaron/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:2189: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)  # noqa: B026


In [29]:
# Et on renomme la colonne du quartier de l'imm ainsi obtenue
df_av_quartiers.rename(columns = {'c_qu':'quartier_imm'}, inplace = True)
# On supprime les colonnes inutiles
df_av_quartiers = df_av_quartiers.drop(['index_left', 'n_sq_qu', 'c_quinsee', 'l_qu', 'c_ar',
       'n_sq_ar', 'perimetre', 'surface'], axis=1)

## 2. On se concentre ensuite sur les adresses du domicile des propriétaires

In [30]:
# On charge le dataframe qu'on vient d'obtenir, mais cette fois les coordonées qui nous intéressent sont celles des domiciles des propriétaires
df_av_quartiers['coord_pers'] = [Point(xy) for xy in zip(df_av_quartiers.lng_pers, df_av_quartiers.lat_pers)]
df_av_quartiers = geopandas.GeoDataFrame(df_av_quartiers, geometry=df_av_quartiers.coord_pers) 

In [31]:
# Récupération de l'information quartier
df_av_quartiers_2 = gdf_quartiers.sjoin(df_av_quartiers, predicate='contains', how='right')

df_av_quartiers_2.rename(columns = {'c_qu':'quartier_pers'}, inplace = True)

df_av_quartiers_2 = df_av_quartiers_2.drop(['index_left', 'n_sq_qu', 'c_quinsee', 'l_qu', 'c_ar',
       'n_sq_ar', 'perimetre', 'surface'], axis=1)

/home/aaron/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:2189: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)  # noqa: B026


In [32]:
# Si on voulait faire un df avec que les propriétaires habitant dans Paris :
# sub_sub_df = df_av_quartiers_2.dropna(subset=['quartier_pers'])

# Etape 5 : sauvegarde des dataframes en csv

Les différents dataframes que nous venons de produire sont donc :
- df_proprios > 1 ligne = 1 proprio (mais n'a pas le nombre total d'immeubles par personne)
- df_bis > 1 ligne = 1 code. (n'a pas encore les bons noms de colonnes)
- df_ter > 1 ligne = 1 immeuble
- geoloc > apres l'entreprise de geolocalisation
- df_av_quartiers > avec le quartier mais que de imm
- df_av_quartiers_2 > avec les deux quartiers

Nous nous intéressons donc surtout à `df_av_quartiers_2`.

Nous allons dans un premier temps le manipulier pour obtenir un dataframe ou une ligne = un proprio, mais qui contient toutes les informations que nous avons associées au dataframe final. Nous allons dans un second temps exporter ces dataframes

In [33]:
# Création du dataframe indexé par rapport aux proprios, mais avec toutes les informations utiles
df_proprios_propre = df_av_quartiers_2.drop_duplicates('index_total')

In [34]:
print(f"Nous avons donc comme nombre total d'immeubles :\n- avant les traitements : {len(df_ter)}\n- apres les traitements : {len(df_av_quartiers_2)}")

Nous avons donc comme nombre total d'immeubles :
- avant les traitements : 2118
- apres les traitements : 2118


In [37]:
df_proprios_propre = df_proprios_propre[['index_total', 'nom_pers', 'prenom_pers', 'civilite_pers', 'org', 'num_pers',
        'type_voie_pers', 'nom_voie_pers', 'arr_pers', 'ville_pers', 'loc_pers', 'part_pers', 'codes', 'nb_codes',
        'nveaux_codes', 'nb_imms', 'lat_pers', 'lng_pers', 'result_type_pers', 'result_city_pers', 'geometry', 'coord_pers',
        'quartier_imm', 'quartier_pers']]

In [38]:
print(f"Nombre de lignes de df_proprios : {len(df_proprios)}")
print(f"Nombre de lignes de df_bis : {len(df_bis)}")
print(f"Nombre de lignes de df_ter : {len(df_ter)}")
print(f"Nombre de lignes de geoloc : {len(geoloc)}")
print(f"Nombre de lignes de df_av_quartiers : {len(df_av_quartiers)}")
print(f"Nombre de lignes de df_av_quartiers_2 : {len(df_av_quartiers_2)}")
print(f"Nombre de lignes de df_proprios_propre : {len(df_proprios_propre)}")

Nombre de lignes de df_proprios : 1051
Nombre de lignes de df_bis : 1657
Nombre de lignes de df_ter : 2118
Nombre de lignes de geoloc : 2118
Nombre de lignes de df_av_quartiers : 2118
Nombre de lignes de df_av_quartiers_2 : 2118
Nombre de lignes de df_proprios_propre : 1051


In [40]:
# Définition des chemins utiles
folder_path = os.path.join(parent_dir, 'sample_1951')
folder_path_sauvegarde = os.path.join(folder_path, 'df_sauvegarde')

os.mkdir(folder_path_sauvegarde)

# Sauvegarde des dataframes

# Les étapes intermédiaires 
df_proprios.to_csv(os.path.join(folder_path_sauvegarde, 'df_par_proprio_debut_traitement_1951.csv'))
df_bis.to_csv(os.path.join(folder_path_sauvegarde, 'df_par_code_1951.csv'))
df_ter.to_csv(os.path.join(folder_path_sauvegarde, 'df_par_num_avant_geoloc_1951.csv'))
geoloc.to_csv(os.path.join(folder_path_sauvegarde, 'df_par_num_apres_geoloc_1951.csv'))
df_av_quartiers.to_csv(os.path.join(folder_path_sauvegarde, 'df_par_num_av_quartier_imm_1951.csv'))

# Et les deux tableaux finaux : le contenu de l'annuaire indexé par propriétaire, et le contenu de l'annaiare indexé par immeuble parisien
df_av_quartiers_2.to_csv(os.path.join(folder_path, 'df_par_num_complet_1951.csv'))
df_proprios_propre.to_csv(os.path.join(folder_path, 'df_par_proprio_complet_1951.csv'))